Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
import ast
from collections import Counter

Load dataset

In [ ]:
jobs_file = "real_dataset_android.csv"
df = pd.read_csv(jobs_file)
pd.set_option('display.max_columns', None) 

# Data Exploration

Display basic information about the dataset

In [ ]:
df.head()  # Preview first few rows

In [ ]:
df.info()  # Overview of dataset structure

In [ ]:
df.isna().sum() # Count of missing values per column
# df.isnull().sum()

In [ ]:
df['client_location'].unique()

In [ ]:
df.work_hours.value_counts()

In [ ]:
df.duration.value_counts()

In [ ]:
df.proposals.value_counts()

In [ ]:
df.experience_level.value_counts()

In [ ]:
df.job_type.value_counts()

In [ ]:
df.location.value_counts()

# Data Preprocessing

In [ ]:
print(f'Number of Records: {df.shape[0]}')
df = df.dropna(subset='title')
df = df.drop_duplicates(subset='title', keep='first')
print(f'Number of Records (after dropping NaNs and duplicates): {df.shape[0]}')

Drop unnecessary columns

In [ ]:
df.drop(columns=['location', 'client_industry', 'client_type'], inplace=True)

Drop duplicates and clean missing values

In [ ]:
df.dropna(subset=["title"], inplace=True)  # Remove rows where title is missing
df.drop_duplicates(subset=["title"], inplace=True)  # Remove duplicate job titles
df = df.dropna(subset=['min_budget', 'client_location', 'proposals','interviewing', 'invites_sent', 'unanswered_invites']).reset_index(drop=True)

Filter out inconsistent job records

In [ ]:
df = df[~((df['work_hours'].isna()) & 
          ~(df['max_budget'].isna() & df['duration'].isna() & df['work_hours'].isna() & 
            (df['job_type'] == 'Fixed-price')))]

Drop rows where all specified columns have missing values

In [ ]:
df.dropna(subset=['min_budget', 'fixed_price'], how='all', inplace=True)
df.dropna(subset=['proposals', 'interviewing', 'invites_sent', 'unanswered_invites'], how='all', inplace=True)
df.dropna(subset=['interviewing'], inplace=True)

Convert budget columns from string to numeric

In [ ]:
df['min_budget'] = df['min_budget'].replace('[\$,]', '', regex=True).astype(float)
df['max_budget'] = df['max_budget'].replace('[\$,]', '', regex=True).astype(float)

Function to calculate average budget

In [ ]:
def calculate_average(row):
    """
    Calculates the average budget for a job post.

    Args:
        row (pd.Series): A row of the DataFrame containing min and max budgets.

    Returns:
        float: The average budget if max_budget exists; otherwise, min_budget.
    """
    if pd.notna(row['max_budget']):
        return (row['min_budget'] + row['max_budget']) / 2
    return row['min_budget']


In [ ]:
# Apply the function to compute the average budget
df['average_budget'] = df.apply(calculate_average, axis=1)

In [ ]:
df['duration'] = df.apply(lambda data: 'Not Defined' if pd.isna(data['duration']) and data['job_type'] == 'Fixed-price' else data['duration'], axis=1)
df['work_hours'] = df.apply(lambda data: 'Flexible' if pd.isna(data['work_hours']) and data['job_type'] == 'Fixed-price' else data['work_hours'], axis=1)

Mapping categorical values to numerical representations

In [ ]:
work_hours_map = {
    'Less than 30 hrs/week': 'less_than_30',
    'More than 30 hrs/week': 'more_than_30',
    'Flexible': 'flexible'
}

In [ ]:
duration_map = {
    'Not Defined': 0,
    '< 1 month': 1,
    '1-3 months': 2,
    '3-6 months': 3,
    '6+ months': 4
}

df['duration'] = df['duration'].map(duration_map)

In [ ]:
proposal_mapping = {
    'Less than 5': 1,
    '5 to 10': 2,
    '10 to 15': 3,
    '15 to 20': 4,
    '20 to 50': 5,
    '50+': 6
}
df['proposals'] = df['proposals'].map(proposal_mapping)

In [ ]:
experience_mapping = {
    'Entry': 1,
    'Intermediate': 2,
    'Expert': 3
}
df['experience_level'] = df['experience_level'].map(experience_mapping)


In [ ]:
job_type_mapping = {
    'Hourly': 0,
    'Fixed-price': 1
}
df['job_type'] = df['job_type'].map(job_type_mapping)


In [ ]:
country_map = {
    'USA': 'United States', 'GBR': 'United Kingdom', 'CAN': 'Canada', 'TUN': 'Tunisia',
    'ITA': 'Italy', 'FRA': 'France', 'IND': 'India', 'AUS': 'Australia', 'ARE': 'United Arab Emirates',
    'IDN': 'Indonesia', 'SGP': 'Singapore', 'PAK': 'Pakistan', 'PRT': 'Portugal', 'MEX': 'Mexico',
    'GRC': 'Greece', 'BEL': 'Belgium', 'COL': 'Colombia', 'ISR': 'Israel', 'MKD': 'North Macedonia',
    'NGA': 'Nigeria', 'ZAF': 'South Africa', 'MLT': 'Malta', 'LKA': 'Sri Lanka', 'KWT': 'Kuwait',
    'IRL': 'Ireland', 'DEU': 'Germany', 'JOR': 'Jordan', 'MYS': 'Malaysia', 'CHE': 'Switzerland',
    'CYP': 'Cyprus', 'KOR': 'South Korea', 'BRA': 'Brazil', 'SRB': 'Serbia', 'SWE': 'Sweden',
    'PRI': 'Puerto Rico', 'MKD': 'North Macedonia', 'BHR': 'Bahrain', 'TUR': 'Turkey',
    'BOL': 'Bolivia', 'PHL': 'Philippines', 'LUX': 'Luxembourg', 'NEP': 'Nepal', 'ARG': 'Argentina'
}

df['client_location'] = df['client_location'].replace(country_map)

One-hot encoding for work hours category

In [ ]:
df = pd.get_dummies(df, columns=['work_hours'], prefix='work_hours', dtype='int')

Function to convert money values (K, M notation) to numerical format

In [ ]:
def convert_money(value):
    """
    Converts monetary values from string format with 'K' or 'M' to float.

    Args:
        value (str): The monetary value as a string.

    Returns:
        float: The converted monetary value.
    """
    if pd.isna(value): 
        return None
    value = value.replace("$", "")  
    if "K" in value:
        return float(value.replace("K", "")) * 1_000  
    elif "M" in value:
        return float(value.replace("M", "")) * 1_000_000  
    return float(value)

In [ ]:
# Apply money conversion function to the client spending column
df['client_total_spent'] = df['client_total_spent'].apply(convert_money)

In [ ]:
# Drop unnecessary budget columns
df.drop(columns=['min_budget', 'max_budget', 'fixed_price'], inplace=True)

Plot histogram for client_total_spent

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(df["client_total_spent"], bins=30, kde=True, color='blue')
plt.title("Histogram of client_total_spent")

plt.show()

In [ ]:
log_client_total_spent = np.log(df['client_total_spent'])

Plot histogram for log_client_total_spent

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(log_client_total_spent, bins=30, kde=True, color='blue')
plt.title("Histogram of log_client_total_spent")

plt.show()

In [ ]:
log_client_total_spent = log_client_total_spent.fillna(log_client_total_spent.mean())
df['reversed_log_client_total_spent'] = np.exp(log_client_total_spent)

Plot histogram for reversed_log_client_total_spent

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(df["reversed_log_client_total_spent"], bins=30, kde=True, color='blue')
plt.title("Histogram of reversed_log_client_total_spent")

plt.show()

Pick 600 Records from Dataset

In [ ]:
null_rows = df[(df['hires'].isnull()) & (df['active'].isnull())]
rows_to_drop = df.shape[0] - 600
df = df.drop(null_rows.head(rows_to_drop).index).reset_index(drop=True)

Preprocess Skills Column

In [ ]:
df['skills'] = df['skills'].apply(ast.literal_eval)

skill_counts = Counter(skill for skills in df['skills'] for skill in skills)

skill_counts_df = pd.DataFrame(skill_counts.items(), columns=['Skill', 'Count']).sort_values(by='Count', ascending=False)

skill_counts_df.head()

In [ ]:
skill_counts_df.to_csv('skill_count.csv', index=False)

Insert a new column for job category (track)

In [ ]:
track_name = "android_developer"
df.insert(0, "track_name", track_name)

Final Check of Dataset

In [ ]:
df.head()

In [ ]:
df.info(verbose=True)

In [ ]:
df.isnull().sum()

Save CSV File

In [ ]:
track = 'AD'
df.to_csv(f'preprocessed_{track}.csv', index=False)

In [ ]:
df.shape